Basic Statistics:

    Total Kills (K) per player. (By scope, by side, ) 
    Total Deaths (D) per player.
    Kill/Death ratio (K/D) per player.
    Average Combat Score (ACS) per player.
    Average Damage per Round (ADR) per player.

Performance Metrics:

    Headshot Percentage (HS%) per player.
    First Kills (FK) and First Deaths (FD) counts and ratios.
    Kill/Death difference (+/-) per player.
    Kill/Death difference in pistol rounds (+/-) per player.

Agent-Specific Metrics:

    Agent popularity and performance analysis.
    Win rates and performance on specific maps for each agent.

Team Performance:

    Total team Kills, Deaths, and K/D ratio.
    Team Average Combat Score (ACS) and Average Damage per Round (ADR).
    Team First Kills (FK) and First Deaths (FD) counts and ratios.

Map Performance:

    Individual player performance on each map.
    Team performance on each map.
    Win rates on each map.

Stage and Series Analysis:

    Performance in different stages (e.g., Playoffs, Grand Final).
    Series-level statistics and win rates.

Comparative Analysis:

    Compare individual player statistics against the team average.
    Compare agent performance across players.

Trend Analysis:

    Analyze how player performance changes over different maps or series.
    Identify trends in individual player K/D ratios.

Impact Analysis:

    Assess the impact of individual player performance on team success.


In [1]:
import pandas as pd
from utils import *

In [5]:
data = pd.read_csv('Data/stat_match_ascension.csv')
report = number_action_by_player_by_scope(df = data, action='K', average=True, team_name="Gentle Mates", side='Total')

In [17]:
import pandas as pd

df = pd.read_csv('champions-tour-2024-pacific-kickoff_data/general_data_champions-tour-2024-pacific-kickoff.csv')



In [39]:
# Match rate
data = df.drop_duplicates(['Stage', 'Series','Team Name'])
played = data[data['Team Name'] == "Paper Rex"].dropna()
wins = data[data['Team Name'] == "Paper Rex"].where(data['Team Name'] == data['winner']).dropna()
win_rate_matches = round(len(wins) / len(played), 2)

In [40]:
data_round = df.drop_duplicates(['Stage', 'Series','Team Name', 'Map #'])[['Stage', 'Series', 'Map #','Team Name', 'rounds', 'winner']]
data_round.head(2)

,Stage,Series,Map #,Team Name,rounds,winner
0,Group Stage,Opening (A),2,T1,"1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0...",T1
5,Group Stage,Opening (A),2,BLEED,"0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1...",T1


In [41]:
data_round['rounds'] = data_round['rounds'].apply(lambda x: list(map(int, x.split(', '))))
data_round['total_rounds'] = data_round['rounds'].apply(lambda x: sum(x))

In [80]:
def set_map_winner(df):
    
    map_winners = []

    for i in range(0,len(df)-1,2):
        if df['total_rounds'].iloc[i] > df['total_rounds'].iloc[i+1]:
            map_winners.append(df['Team Name'].iloc[i])
            map_winners.append(df['Team Name'].iloc[i])
        else:
            map_winners.append(df['Team Name'].iloc[i+1])
            map_winners.append(df['Team Name'].iloc[i+1])
    
    df['map_winners'] = map_winners

    return df


In [95]:
map_winners = set_map_winner(data_round)
played = map_winners[map_winners['Team Name'] == "Paper Rex"].dropna()
wins = map_winners[map_winners['Team Name'] == "Paper Rex"].where(map_winners['Team Name'] == map_winners['map_winners']).dropna()
win_rate_maps = round(len(wins) / len(played), 3)

In [2]:
data_round = df.drop_duplicates(['Stage', 'Series','Team Name', 'Map #'])[['Stage', 'Series', 'Map #','Team Name', 'rounds', 'winner']]
data_round['rounds'] = data_round['rounds'].apply(lambda x: list(map(int, x.split(', '))))
data_round['total_rounds'] = data_round['rounds'].apply(lambda x: sum(x))

In [20]:
data_round = df.drop_duplicates(['Stage', 'Series','Team Name', 'Map #'])[['Stage', 'Series', 'Map #','Team Name', 'rounds', 'winner']]
data_round['rounds'] = data_round['rounds'].apply(lambda x: list(map(int, x.split(', '))))
data_round['total_rounds'] = data_round['rounds'].apply(lambda x: sum(x))
win_rates = {team : round(sum(data_round[data_round['Team Name'] == team].dropna()['total_rounds']) / data_round[data_round['Team Name'] == team].dropna()['rounds'].apply(len).sum(), 2) for team in list(set(df['Team Name']))}

In [21]:
win_rates

{'Global Esports': 0.43,
 'Talon Esports': 0.48,
 'DetonatioN FocusMe': 0.4,
 'BLEED': 0.44,
 'ZETA DIVISION': 0.48,
 'Rex Regum Qeon': 0.46,
 'T1': 0.57,
 'Team Secret': 0.52,
 'Gen.G': 0.52,
 'DRX': 0.56,
 'Paper Rex': 0.51}

In [11]:
len(data_round[data_round['Team Name'] == 'T1'].dropna()['rounds'][0])

19

In [24]:
def calculate_win_rate(df, scope = 'match'):
    
    match scope:
        case "match":
            data = df.drop_duplicates(['Stage', 'Series','Team Name'])
            win_rates = {team : round(len(data[data['Team Name'] == team].where(data['Team Name'] == data['winner']).dropna()) / len(data[data['Team Name'] == team].dropna()),2) for team in list(set(data['Team Name']))}
            return win_rates
        case "map":
            data_round = df.drop_duplicates(['Stage', 'Series','Team Name', 'Map #'])[['Stage', 'Series', 'Map #','Team Name', 'rounds', 'winner']]
            data_round['rounds'] = data_round['rounds'].apply(lambda x: list(map(int, x.split(', '))))
            data_round['total_rounds'] = data_round['rounds'].apply(lambda x: sum(x))
            map_winners = set_map_winner(data_round)
            win_rates = {team : round(len(map_winners[map_winners['Team Name'] == team].where(map_winners['Team Name'] == map_winners['map_winners']).dropna()) / len(map_winners[map_winners['Team Name'] == team].dropna()),2) for team in list(set(data['Team Name']))}
            return win_rates
        case "round":
            data_round = df.drop_duplicates(['Stage', 'Series','Team Name', 'Map #'])[['Stage', 'Series', 'Map #','Team Name', 'rounds', 'winner']]
            data_round['rounds'] = data_round['rounds'].apply(lambda x: list(map(int, x.split(', '))))
            data_round['total_rounds'] = data_round['rounds'].apply(lambda x: sum(x))
            win_rates = {team : round(sum(data_round[data_round['Team Name'] == team].dropna()['total_rounds']) / data_round[data_round['Team Name'] == team].dropna()['rounds'].apply(len).sum(), 2) for team in list(set(df['Team Name']))}
            return win_rates

In [25]:
win_rate = calculate_win_rate(df, scope = 'round')

In [26]:
win_rate

{'Global Esports': 0.43,
 'Talon Esports': 0.48,
 'DetonatioN FocusMe': 0.4,
 'BLEED': 0.44,
 'ZETA DIVISION': 0.48,
 'Rex Regum Qeon': 0.46,
 'T1': 0.57,
 'Team Secret': 0.52,
 'Gen.G': 0.52,
 'DRX': 0.56,
 'Paper Rex': 0.51}